In [31]:
import pandas as pd
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, BatchNormalization
from datetime import datetime, timedelta,date
from mapper import mapper_ru

In [7]:
path_time_series = './data/COVID-19_plus_Russia/csse_covid_19_data/csse_covid_19_time_series/'
recovered_global_name = 'time_series_covid19_recovered_global.csv'
recovered_ru_name = 'time_series_covid19_confirmed_RU.csv'
deaths_global_name = 'time_series_covid19_deaths_global.csv'
deaths_ru_name = 'time_series_covid19_deaths_RU.csv'
path = './data/'
countries_data_name = 'countries.csv'
russia_regions_name = 'russia_regions.csv'
sample_submission = 'sample_submission.csv'

In [8]:
russia_regions = pd.read_csv(path+russia_regions_name)
recovered_ru = pd.read_csv(path_time_series+ recovered_ru_name)
deaths_ru = pd.read_csv(path_time_series + deaths_ru_name)

In [14]:
def pre_ru(df):
    df = pd.concat([df.iloc[:,6:7], df.iloc[:,11:]],axis=1)
    return df
recovered_ru = pre_ru(recovered_ru)

In [15]:
def get_dataset_ru(df_ru):
    solution_df = pd.DataFrame({'name':df_ru['Province_State'].values})
    date = df_ru.columns.values[1:]
    solution = pd.concat([solution_df,df_ru[date]],axis=1)
    dataset_ru = solution[date].values.T 
    return dataset_ru

In [16]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
    n = sequences.shape[0]
    X, y = list(), list()
    for i in range(n):
        # find the end of this pattern
        end_ix = i + n_steps
    # check if we are beyond the dataset
        if end_ix > n-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [37]:
def build_model_simple(n_steps,n_features):
    model = Sequential()
    model.add(LSTM(85, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(85, activation='relu'))
    model.add(Dense(170,activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    return model

In [38]:
from sklearn.model_selection import TimeSeriesSplit

In [39]:
tscv = TimeSeriesSplit()

n_steps = 7
n_features = 85
model_rec = build_model_simple(n_steps,n_features)

dataset_rec = get_dataset_ru(recovered_ru) 
X_rec,y_rec = split_sequences(dataset_rec,n_steps)
i = 0
scores = []
for train_index, test_index in tscv.split(X_rec):
    i+=1
    print(f' tscv{i}')
    X_train, X_test = X_rec[train_index], X_rec[test_index]
    y_train, y_test = y_rec[train_index], y_rec[test_index]
    model_tscv = build_model_simple(n_steps,n_features)
    model_tscv.fit(X_train, y_train, epochs=100, verbose=0)
    score = model_tscv.evaluate(X_test,y_test)
    scores.append(score)
    
print(np.mean(scores))

 tscv1
14/14 [==============================] - 4s 320ms/step
 tscv2
14/14 [==============================] - 5s 327ms/step
 tscv3
14/14 [==============================] - 5s 339ms/step
 tscv4
14/14 [==============================] - 5s 343ms/step
 tscv5
14/14 [==============================] - 9s 664ms/step
364361.1150873661


In [8]:
n_steps = 3
n_features = 85
model_rec = build_model_simple(n_steps,n_features)

dataset_rec = get_dataset_ru(recovered_ru) 
X_rec,y_rec = split_sequences(dataset_rec,n_steps)
hisoty_rec = model_rec.fit(X_rec, y_rec, epochs=10, verbose=1)


Epoch 1/10
93/93 [==============================] - 8s 83ms/step - loss: 1461074.1962
Epoch 2/10
93/93 [==============================] - 0s 2ms/step - loss: 1404192.0811
Epoch 3/10
93/93 [==============================] - 0s 2ms/step - loss: 1372711.9234
Epoch 4/10
93/93 [==============================] - 0s 2ms/step - loss: 1353051.4603
Epoch 5/10
93/93 [==============================] - 0s 2ms/step - loss: 1296284.4993
Epoch 6/10
93/93 [==============================] - 0s 2ms/step - loss: 1201192.9086
Epoch 7/10
93/93 [==============================] - 0s 2ms/step - loss: 1042333.1156
Epoch 8/10
93/93 [==============================] - 0s 2ms/step - loss: 884438.8306
Epoch 9/10
93/93 [==============================] - 0s 2ms/step - loss: 785047.2648
Epoch 10/10
93/93 [==============================] - 0s 2ms/step - loss: 629469.6881


In [10]:
columns_ru = recovered_ru['Province_State']
last_date = pd.to_datetime(recovered_ru.columns.values[-1:]).date[0]

In [9]:

n_steps = 3
n_features = 85
model_deaths = build_model_simple(n_steps,n_features)

dataset_deaths = get_dataset_ru(deaths_ru) 
X_deaths,y_deaths = split_sequences(dataset_deaths,n_steps)
hisoty_deaths = model_deaths.fit(X_deaths, y_deaths, epochs=10, verbose=1)

Epoch 1/10
93/93 [==============================] - 7s 73ms/step - loss: 111.0129
Epoch 2/10
93/93 [==============================] - 0s 2ms/step - loss: 102.5468
Epoch 3/10
93/93 [==============================] - 0s 2ms/step - loss: 96.6347
Epoch 4/10
93/93 [==============================] - 0s 2ms/step - loss: 89.0841
Epoch 5/10
93/93 [==============================] - 0s 2ms/step - loss: 80.3686
Epoch 6/10
93/93 [==============================] - 0s 2ms/step - loss: 70.0201
Epoch 7/10
93/93 [==============================] - 0s 2ms/step - loss: 58.8572
Epoch 8/10
93/93 [==============================] - 0s 2ms/step - loss: 44.9057
Epoch 9/10
93/93 [==============================] - 0s 2ms/step - loss: 31.6205
Epoch 10/10
93/93 [==============================] - 0s 2ms/step - loss: 20.3481


In [16]:
class Prediction:
    def __init__(self, model,last_date,columns_df):
        self.columns_df = columns_df
        self.model = model
        self.last_date = last_date
        self.predion_date = self.__make_date_predition()
        self.size_prediction = len(self.predion_date)
        
    def predict(self,dataset):
        self.dateset = dataset
        matrix_prediction = self.__make_prediction_matrix()
        return self.__make_dataframe_prediction(matrix_prediction)
    
    def __make_dataframe_prediction(self,matrix_prediction):
        df = pd.DataFrame(matrix_prediction,columns=self.columns_df)
        df['date'] = self.predion_date
        return df
    def __make_prediction_matrix(self):
        dataset = self.dateset
        model = self.model
        count = self.size_prediction
        
        x = np.array([dataset[-3:,:]])
        x_i = dataset[-2:,:]
        result = []
        for _ in range(count):
            y_pred = model.predict(x)[0]
    #         y_pred_ = list(np.sign(y_pred)*(y_pred))
            y_pred_ = list((y_pred))
            result.append(y_pred_)
            y_pred = y_pred.reshape(1,85)
            x = np.vstack((x_i,y_pred))
            x_i = x[-2:,:]
            x = np.array([x])
        return result
        
    def __make_date_predition(self):
        last_date = self.last_date
        last_date += timedelta(days=1)
        res_date = []
        while last_date< date(2021,1,1):
            res_date.append(last_date)
            last_date += timedelta(days=1)
        return res_date
        
        
    
class Submission:
    def __init__(self,columns_df,date_prediction):
        self.columns_df = columns_df
        self.date_prediction = date_prediction
        
    def submit(self,rec_df,deaths_df,mapper):
        df_res = pd.DataFrame({}) 
        for column in self.columns_df:
            df_res_ = pd.DataFrame({}) 
            df_res_['date'] = self.date_prediction
            df_res_['region'] = [column]*len(self.date_prediction)
            df_res_['prediction_confirmed'] = rec_df[column]
            df_res_['prediction_deaths'] = deaths_df[column]
            
            df_res = pd.concat([df_res,df_res_],axis=0)
        df_res.index=np.arange(df_res.shape[0])
        df_res.region = df_res.region.map(mapper)
        return df_res

In [17]:
predictioner_rec = Prediction(model_rec,last_date,columns_ru)
df_pred_rec = predictioner_rec.predict(dataset_rec)

In [18]:
predictioner_deaths = Prediction(model_deaths,last_date,columns_ru)
df_pred_deaths = predictioner_deaths.predict(dataset_deaths)

In [19]:
submissioner = Submission(columns_ru,predictioner_rec.predion_date)
subm = submissioner.submit(df_pred_rec,df_pred_deaths,mapper_ru)

In [21]:
subm.shape

(21165, 4)

In [22]:
subm.to_csv('subm.csv')